In [1]:
!pip install transformers==4.44.2 joblib==1.4.2 scikit-learn==1.6.0 numpy==1.26.4 pandas==2.2.3 scipy==1.13.1 seaborn==0.13.2 tqdm==4.66.5 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 50.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 94.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 43.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting u

In [2]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/pampa-set/Train.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/pampa-set/Test.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [4]:
tokenizer = AutoTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
model = AutoModelForSequenceClassification.from_pretrained('seyonec/ChemBERTa-zinc-base-v1', num_labels=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Custom dataset class
class SMILESDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=325):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        smiles = self.dataframe.iloc[idx]['SMILES']
        permeability = self.dataframe.iloc[idx]['Permeability']
        inputs = self.tokenizer(smiles, return_tensors='pt', padding="max_length", truncation=True, max_length=self.max_length)
        
        input_ids = inputs['input_ids'].squeeze(0)  # Shape: (sequence_length,)
        attention_mask = inputs['attention_mask'].squeeze(0)  # Shape: (sequence_length,)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(permeability, dtype=torch.float)
        }


# datasets
train_dataset = SMILESDataset(train_df, tokenizer)
test_dataset = SMILESDataset(test_df, tokenizer)
batch_size = 16
# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/179M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 20

In [7]:
# Training loop
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Entered Epoch {epoch + 1}")
    model.train()
    train_loss = 0

    for batch in tqdm(train_loader, desc=f'Training Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        optimizer.zero_grad()

        # Move all batch tensors to device
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"].unsqueeze(1)  # still shape: (batch_size, 1)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels
        )
        loss = outputs.loss
        train_loss += loss.item()

        # Backprop and optimizer step
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}')

Entered Epoch 1


Training Epoch 1/20: 100%|██████████| 348/348 [02:34<00:00,  2.26batch/s]


Epoch 1/20 - Train Loss: 0.8152
Entered Epoch 2


Training Epoch 2/20: 100%|██████████| 348/348 [02:47<00:00,  2.08batch/s]


Epoch 2/20 - Train Loss: 0.4206
Entered Epoch 3


...

Epoch 18/20 - Train Loss: 0.1645
Entered Epoch 19


Training Epoch 19/20: 100%|██████████| 348/348 [02:54<00:00,  2.00batch/s]


Epoch 19/20 - Train Loss: 0.1553
Entered Epoch 20


Training Epoch 20/20: 100%|██████████| 348/348 [02:53<00:00,  2.00batch/s]

Epoch 20/20 - Train Loss: 0.1479


In [8]:
model_name = 'ChemBERTa_model_1_pampa'
model_save_path = f'/kaggle/working/{model_name}'
os.makedirs(model_save_path, exist_ok=True)

tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

print(f'Model and tokenizer saved to {model_save_path}')

Model and tokenizer saved to /kaggle/working/ChemBERTa_model_1_pampa


In [9]:
from scipy.stats import pearsonr, spearmanr

model.eval()
test_loss = 0
test_true_labels = []
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing', unit='batch'):
      
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].unsqueeze(1).to(device).float()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        test_loss += loss.item()

        test_true_labels.extend(labels.cpu().numpy())
        preds = outputs.logits.squeeze().cpu().numpy()  
        predictions.extend(preds)

# Final test loss
avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss:.4f}')

test_true_labels = np.array(test_true_labels).flatten()
predictions = np.array(predictions)
print(test_true_labels.shape)
print(predictions.shape)

# Performance metrics
mse = mean_squared_error(test_true_labels, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true_labels, predictions)
r2 = r2_score(test_true_labels, predictions)
PCC,_ = pearsonr(test_true_labels, predictions)
SCC,_ = spearmanr(test_true_labels, predictions)
# Print performance metrics
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')
print(f'R^2 Score: {r2:.4f}')
print(f'Pearson Correlation Coefficient: {PCC:.4f}')
print(f'Spearman Correlation Coefficient: {SCC:.4f}')

# Print hyperparameters
print("Hyperparameters:")
print(f"Learning Rate: {5e-5}")
print(f"Batch Size: 16")
print(f"Epochs: {num_epochs}")

Testing: 100%|██████████| 87/87 [00:15<00:00,  5.46batch/s]


Test Loss: 0.2910
(1392,)
(1392,)
Mean Squared Error: 0.2910
Root Mean Squared Error: 0.5394
Mean Absolute Error: 0.3874
R^2 Score: 0.5402
Pearson Correlation Coefficient: 0.7778
Spearman Correlation Coefficient: 0.7643
Hyperparameters:
Learning Rate: 5e-05
Batch Size: 16
Epochs: 20


In [11]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model_name = 'ChemBERTa_model_1_pampa'
model_save_path = f'/kaggle/working/{model_name}'

if not os.path.exists(model_save_path):
    raise FileNotFoundError(f"The model directory {model_save_path} does not exist.")

tokenizer = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModel.from_pretrained(model_save_path).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at /kaggle/working/ChemBERTa_model_1_pampa and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Load your datasets
train_df = pd.read_csv('/kaggle/input/pampa-set/Train.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/pampa-set/Test.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [13]:
train_encodings = tokenizer(list(train_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")
test_encodings = tokenizer(list(test_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")

In [14]:
from tqdm import tqdm 
batch_size = 16 

def generate_embeddings(encodings, batch_size):
    embeddings = []
    model.eval() 
    with torch.no_grad():
        for i in tqdm(range(0, len(encodings['input_ids']), batch_size), desc="Processing batches"):
            batch = {key: val[i:i + batch_size].to(device) for key, val in encodings.items()}  
            outputs = model(**batch)
            embeddings.append(outputs.last_hidden_state)
    return torch.cat(embeddings, dim=0)

In [15]:
train_embeddings = generate_embeddings(train_encodings, batch_size)
print(train_embeddings.shape)
train_embeddings = torch.mean(train_embeddings, dim=1)
print(train_embeddings.shape)

Processing batches: 100%|██████████| 348/348 [00:34<00:00, 10.20it/s]


torch.Size([5568, 237, 768])
torch.Size([5568, 768])


In [16]:
column_names = [f'x_fine_emb_ChemBerta{i}' for i in range(train_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=train_embeddings.cpu().numpy(), columns=column_names)
train_data = pd.concat([train_df, embeddings_df], axis=1)

In [17]:
test_embeddings = generate_embeddings(test_encodings, batch_size)
print(test_embeddings.shape)
test_embeddings = torch.mean(test_embeddings, dim=1)
print(test_embeddings.shape)

Processing batches: 100%|██████████| 87/87 [00:08<00:00,  9.83it/s]


torch.Size([1392, 237, 768])
torch.Size([1392, 768])


In [18]:
column_names = [f'x_fine_emb_ChemBerta{i}' for i in range(test_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=test_embeddings.cpu().numpy(), columns=column_names)
test_data = pd.concat([test_df, embeddings_df], axis=1)

In [21]:
train_data.to_csv("/kaggle/working/Train_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_pampa.csv",index=False)
test_data.to_csv("/kaggle/working/Test_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_pampa.csv",index=False)

In [22]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [23]:
train_data = pd.read_csv("/kaggle/working/Train_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_pampa.csv")
test_data = pd.read_csv("/kaggle/working/Test_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_pampa.csv")

In [24]:
def train_and_test_predict(models, X_train, y_train, X_test, y_test):
    kf = KFold(n_splits=5, shuffle=True, random_state=101)
    results = {}
    predictions = []  

    for model in models:
        model_name = model.__class__.__name__
        predictions_train = []
        actual_y_train = []

        test_predictions_folds = []

        

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)

            y_pred_fold = model.predict(X_val_fold)
            y_pred_fold = np.clip(y_pred_fold, -10, -3.9)
            predictions_train.extend(y_pred_fold)
            actual_y_train.extend(y_val_fold)

            predictions_test_fold = model.predict(X_test)
            predictions_test_fold = np.clip(predictions_test_fold, -10, -3.9)
            test_predictions_folds.append(predictions_test_fold)


        mse_train = mean_squared_error(actual_y_train, predictions_train)
        mae_train = mean_absolute_error(actual_y_train, predictions_train)
        rmse_train = np.sqrt(mse_train)
        r2_train = r2_score(actual_y_train, predictions_train)
        pearson_train, _ = pearsonr(actual_y_train, predictions_train)
        spearman_train, _ = spearmanr(actual_y_train, predictions_train)


        predictions_test_mean = np.mean(test_predictions_folds, axis=0)
        predictions_test_std = np.std(test_predictions_folds, axis=0)

        mse_test = mean_squared_error(y_test, predictions_test_mean)
        mae_test = mean_absolute_error(y_test, predictions_test_mean)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_test, predictions_test_mean)
        print(r2_test)
        pearson_test, _ = pearsonr(y_test, predictions_test_mean)
        spearman_test, _ = spearmanr(y_test, predictions_test_mean)
        
        

        predictions.append({
            'Model': model_name,
            'Y Train pred': predictions_train,
            'Y Test actual': y_test,
            'Test prediction folds': test_predictions_folds,
            'Test Predictions Mean': predictions_test_mean,
            'Test Predictions Std': predictions_test_std,

        })

        results[model_name] = {
            'Train MSE (5 fold cv)': f"{mse_train:.4f}",
            'Train MAE (5 fold cv)': f"{mae_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train R2 (5 fold cv)': f"{r2_train:.4f}",
            'Train PCC (5 fold cv)': f"{pearson_train:.4f}",
            'Train SCC (5 fold cv)': f"{spearman_train:.4f}",
            'Test MSE': f"{mse_test:.4f}",
            'Test MAE': f"{mae_test:.4f}",
            'Test RMSE': f"{rmse_test:.4f}",
            'Test R2': f"{r2_test:.4f}",
            'Test Pearson Correlation': f"{pearson_test:.4f}",
            'Test Spearman Correlation': f"{spearman_test:.4f}",
        }

    results_df = pd.DataFrame(results).T
    predictions_df = pd.DataFrame(predictions)

    return results_df, predictions_df



In [25]:
X_train = train_data.drop(['ID','SMILES','Permeability'],axis=1)
y_train = train_data['Permeability']
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

X_test = test_data.drop(['ID','SMILES','Permeability'],axis=1)
y_test = test_data['Permeability']
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
models = [
    lgb.LGBMRegressor(objective='regression',metric='rmse',boosting_type='gbdt',num_leaves=31,learning_rate=0.05,random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(), 
    KNeighborsRegressor(n_neighbors=3),
    SVR(),  
    MLPRegressor(random_state=101)
]
result_df, prediction_df = train_and_test_predict(models, X_train,y_train, X_test,  y_test)
result_df

X_train shape:  (5568, 768)
y_train shape:  (5568,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X_test shape:  (1392, 768)
y_test shape:  (1392,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.075394 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 4454, number of used features: 768
[LightGBM] [Info] Start training from score -5.738310
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028208 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 4454, number of used features: 768
[LightGBM] [Info] Start training from score -5.742699
[LightGBM] [Info] Auto-choosing col-wise multi-threading

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.1102,0.2400,0.3320,0.8231,0.9073,0.8907,0.2505,0.3567,0.5005,0.6042,0.7811,0.7656
DecisionTreeRegressor,0.2224,0.3424,0.4716,0.6429,0.8233,0.7986,0.2821,0.3819,0.5311,0.5543,0.7559,0.7364
RandomForestRegressor,0.1127,0.2427,0.3357,0.8190,0.9051,0.8859,0.2528,0.3575,0.5028,0.6004,0.7795,0.7627
GradientBoostingRegressor,0.1083,0.2394,0.3290,0.8262,0.9089,0.8918,0.2535,0.3596,0.5035,0.5995,0.7785,0.7625
AdaBoostRegressor,0.1330,0.2801,0.3647,0.7865,0.8930,0.8715,0.2605,0.3806,0.5104,0.5884,0.7708,0.7537
XGBRegressor,0.1317,0.2600,0.3629,0.7885,0.8896,0.8704,0.2532,0.3562,0.5032,0.5998,0.7799,0.7619
ExtraTreesRegressor,0.1249,0.2516,0.3534,0.7995,0.8949,0.8761,0.2572,0.3593,0.5071,0.5936,0.7765,0.7585
LinearRegression,0.1264,0.2591,0.3555,0.7971,0.8942,0.8813,0.2537,0.3588,0.5037,0.5991,0.7788,0.7646
KNeighborsRegressor,0.1402,0.2726,0.3744,0.7749,0.8821,0.8567,0.2643,0.3652,0.5141,0.5823,0.7705,0.7493
SVR,0.1073,0.2352,0.3276,0.8277,0.9099,0.8933,0.2504,0.3545,0.5004,0.6043,0.7815,0.7681


In [26]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.1102,0.2400,0.3320,0.8231,0.9073,0.8907,0.2505,0.3567,0.5005,0.6042,0.7811,0.7656
DecisionTreeRegressor,0.2224,0.3424,0.4716,0.6429,0.8233,0.7986,0.2821,0.3819,0.5311,0.5543,0.7559,0.7364
RandomForestRegressor,0.1127,0.2427,0.3357,0.8190,0.9051,0.8859,0.2528,0.3575,0.5028,0.6004,0.7795,0.7627
GradientBoostingRegressor,0.1083,0.2394,0.3290,0.8262,0.9089,0.8918,0.2535,0.3596,0.5035,0.5995,0.7785,0.7625
AdaBoostRegressor,0.1330,0.2801,0.3647,0.7865,0.8930,0.8715,0.2605,0.3806,0.5104,0.5884,0.7708,0.7537
XGBRegressor,0.1317,0.2600,0.3629,0.7885,0.8896,0.8704,0.2532,0.3562,0.5032,0.5998,0.7799,0.7619
ExtraTreesRegressor,0.1249,0.2516,0.3534,0.7995,0.8949,0.8761,0.2572,0.3593,0.5071,0.5936,0.7765,0.7585
LinearRegression,0.1264,0.2591,0.3555,0.7971,0.8942,0.8813,0.2537,0.3588,0.5037,0.5991,0.7788,0.7646
KNeighborsRegressor,0.1402,0.2726,0.3744,0.7749,0.8821,0.8567,0.2643,0.3652,0.5141,0.5823,0.7705,0.7493
SVR,0.1073,0.2352,0.3276,0.8277,0.9099,0.8933,0.2504,0.3545,0.5004,0.6043,0.7815,0.7681


In [27]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-6.993364332761411, -6.86570211950116, -6.992...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.00699791992174, -6.42197364411304, -5.415...","[-6.023240802020441, -6.3837454517624845, -5.4...","[0.06768960883259488, 0.06799019277728097, 0.0..."
1,DecisionTreeRegressor,"[-6.958429709, -7.0, -7.0, -5.98, -6.03, -5.33...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.64, -6.56, -5.62, -6.22, -4.57, -6.74, -6...","[-5.9009666214, -6.456, -5.499999999999999, -5...","[0.21028535378408247, 0.29890466707631036, 0.3..."
2,RandomForestRegressor,"[-6.94925352888, -6.883499021390004, -7.042611...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.040009647450001, -6.468105661370001, -5.4...","[-6.0225312440789995, -6.468827848913001, -5.5...","[0.04224245560360799, 0.0959394686106434, 0.04..."
3,GradientBoostingRegressor,"[-6.957541922836725, -6.824154950907995, -6.94...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.067699647568223, -6.5136458071144405, -5....","[-6.075616231293237, -6.523390611244184, -5.47...","[0.03546179410469545, 0.0423814019204641, 0.06..."
4,AdaBoostRegressor,"[-6.718596073962517, -6.824987027054409, -6.91...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.176476420528, -6.372833286146339, -5.7491...","[-6.301911325140528, -6.391600526863818, -5.71...","[0.06643719627453003, 0.03643862143635866, 0.0..."
5,XGBRegressor,"[-6.9519024, -6.884763, -6.793566, -6.1455474,...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.112988, -6.4744864, -5.689469, -5.9820952...","[-6.0297685, -6.4334135, -5.486541, -6.0469465...","[0.06496624, 0.34157488, 0.12392911, 0.0837470..."
6,ExtraTreesRegressor,"[-6.961269660760001, -6.906366825550001, -7.01...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.041344143625002, -6.384602691660004, -5.5...","[-6.0149621177800014, -6.365625530444004, -5.4...","[0.030190983078895648, 0.0830521096002978, 0.0..."
7,LinearRegression,"[-6.860693672830814, -6.701425363651921, -7.08...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.75772812208495, -6.563853824668913, -5.55...","[-5.861653038246806, -6.422272056856061, -5.59...","[0.06909987747812797, 0.15352410271796044, 0.1..."
8,KNeighborsRegressor,"[-7.0, -6.973333333333334, -6.986666666666667,...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.956666666666666, -6.266666666666667, -5.2...","[-5.998666666666667, -6.243333333333333, -5.33...","[0.1339253523422657, 0.07229568912920473, 0.03..."
9,SVR,"[-6.981122611971183, -6.859227098270428, -6.97...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.8833460733451775, -6.248129196188305, -5....","[-5.924039584273174, -6.23306345619096, -5.483...","[0.061639738983368664, 0.14028402221988104, 0...."


In [28]:
result_df.to_csv('/kaggle/working/Results_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_pampa.csv')
prediction_df.to_csv('/kaggle/working/Prediction_data_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_pampa.csv')

In [29]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.1102,0.2400,0.3320,0.8231,0.9073,0.8907,0.2505,0.3567,0.5005,0.6042,0.7811,0.7656
DecisionTreeRegressor,0.2224,0.3424,0.4716,0.6429,0.8233,0.7986,0.2821,0.3819,0.5311,0.5543,0.7559,0.7364
RandomForestRegressor,0.1127,0.2427,0.3357,0.8190,0.9051,0.8859,0.2528,0.3575,0.5028,0.6004,0.7795,0.7627
GradientBoostingRegressor,0.1083,0.2394,0.3290,0.8262,0.9089,0.8918,0.2535,0.3596,0.5035,0.5995,0.7785,0.7625
AdaBoostRegressor,0.1330,0.2801,0.3647,0.7865,0.8930,0.8715,0.2605,0.3806,0.5104,0.5884,0.7708,0.7537
XGBRegressor,0.1317,0.2600,0.3629,0.7885,0.8896,0.8704,0.2532,0.3562,0.5032,0.5998,0.7799,0.7619
ExtraTreesRegressor,0.1249,0.2516,0.3534,0.7995,0.8949,0.8761,0.2572,0.3593,0.5071,0.5936,0.7765,0.7585
LinearRegression,0.1264,0.2591,0.3555,0.7971,0.8942,0.8813,0.2537,0.3588,0.5037,0.5991,0.7788,0.7646
KNeighborsRegressor,0.1402,0.2726,0.3744,0.7749,0.8821,0.8567,0.2643,0.3652,0.5141,0.5823,0.7705,0.7493
SVR,0.1073,0.2352,0.3276,0.8277,0.9099,0.8933,0.2504,0.3545,0.5004,0.6043,0.7815,0.7681


In [30]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-6.993364332761411, -6.86570211950116, -6.992...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.00699791992174, -6.42197364411304, -5.415...","[-6.023240802020441, -6.3837454517624845, -5.4...","[0.06768960883259488, 0.06799019277728097, 0.0..."
1,DecisionTreeRegressor,"[-6.958429709, -7.0, -7.0, -5.98, -6.03, -5.33...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.64, -6.56, -5.62, -6.22, -4.57, -6.74, -6...","[-5.9009666214, -6.456, -5.499999999999999, -5...","[0.21028535378408247, 0.29890466707631036, 0.3..."
2,RandomForestRegressor,"[-6.94925352888, -6.883499021390004, -7.042611...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.040009647450001, -6.468105661370001, -5.4...","[-6.0225312440789995, -6.468827848913001, -5.5...","[0.04224245560360799, 0.0959394686106434, 0.04..."
3,GradientBoostingRegressor,"[-6.957541922836725, -6.824154950907995, -6.94...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.067699647568223, -6.5136458071144405, -5....","[-6.075616231293237, -6.523390611244184, -5.47...","[0.03546179410469545, 0.0423814019204641, 0.06..."
4,AdaBoostRegressor,"[-6.718596073962517, -6.824987027054409, -6.91...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.176476420528, -6.372833286146339, -5.7491...","[-6.301911325140528, -6.391600526863818, -5.71...","[0.06643719627453003, 0.03643862143635866, 0.0..."
5,XGBRegressor,"[-6.9519024, -6.884763, -6.793566, -6.1455474,...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.112988, -6.4744864, -5.689469, -5.9820952...","[-6.0297685, -6.4334135, -5.486541, -6.0469465...","[0.06496624, 0.34157488, 0.12392911, 0.0837470..."
6,ExtraTreesRegressor,"[-6.961269660760001, -6.906366825550001, -7.01...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-6.041344143625002, -6.384602691660004, -5.5...","[-6.0149621177800014, -6.365625530444004, -5.4...","[0.030190983078895648, 0.0830521096002978, 0.0..."
7,LinearRegression,"[-6.860693672830814, -6.701425363651921, -7.08...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.75772812208495, -6.563853824668913, -5.55...","[-5.861653038246806, -6.422272056856061, -5.59...","[0.06909987747812797, 0.15352410271796044, 0.1..."
8,KNeighborsRegressor,"[-7.0, -6.973333333333334, -6.986666666666667,...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.956666666666666, -6.266666666666667, -5.2...","[-5.998666666666667, -6.243333333333333, -5.33...","[0.1339253523422657, 0.07229568912920473, 0.03..."
9,SVR,"[-6.981122611971183, -6.859227098270428, -6.97...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.8833460733451775, -6.248129196188305, -5....","[-5.924039584273174, -6.23306345619096, -5.483...","[0.061639738983368664, 0.14028402221988104, 0...."
